In [ ]:
!pip install ortoolpy

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ortoolpy

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals

In [ ]:
df_tc = pd.read_csv("../input/100honch7/trans_cost.csv", index_col="工場")
df_demand = pd.read_csv("../input/100honch7/demand.csv")
df_supply = pd.read_csv("../input/100honch7/supply.csv")

In [ ]:
np.random.seed(1) #np.randomで乱数を生成する時のseed値を1に設定している。つまり、次回以降np.random.rand()で乱数を出力すると、常にseed=1に対応する同じ乱数が出力される
nw = len(df_tc.index)
nf = len(df_tc.columns)
pr = list(product(range(nw), range(nf))) #prodctは、複数の集合に対して、その直積を返す。

In [ ]:
#pythonの数理モデル分析について参考になるサイト : https://docs.pyq.jp/python/math_opt/pulp.html

m1 = model_min() #　model_min()は、ある目的関数に対して制約条件のもとで最小化を行うモデル

v1 = {(i,j): LpVariable('v%d_%d'%(i,j), lowBound=0) for i,j in pr} #下限値を0としてもつ、i*j個の変数{vi_j}を生成
m1 += lpSum(df_tc.iloc[i][j]*v1[i,j] for i,j in pr) # m1 += の『+=』は、モデルm1に目的関数を設定するという意味なので注意。また、lpsumはsumよりも高速に変数和の処理を行う

In [ ]:
#m1に対して追加の制約条件を与える

for i in range(nw):
    m1 += lpSum(v1[i,j] for j in range(nf)) <= df_supply.iloc[0][i]
for j in range(nf):
    m1 += lpSum(v1[i,j] for i in range(nw)) >= df_demand.iloc[0][j]

In [ ]:
m1.solve()

In [ ]:
df_tr_sol = df_tc.copy()
total_cost = 0

for k,x in v1.items():
    i,j = k[0],k[1]
    df_tr_sol.iloc[i][j] = value(x)
    total_cost += df_tc.iloc[i][j]*value(x)
    
print(df_tr_sol)
print("総輸送コスト:" + str(total_cost))

In [ ]:
#最適輸送ルートをグラフで可視化する

import networkx as nx
import matplotlib.pyplot as plt

df_tr = df_tr_sol.copy()
df_pos = pd.read_csv("../input/100honch7/trans_route_pos.csv")

#グラフオブジェクトの生成
G = nx.Graph()

#頂点の設定
for i in range(len(df_pos.columns)):
    G.add_node(df_pos.columns[i])

#辺の設定、エッジの重みのリスト化

num_pre = 0
edge_weights = []
size = 0.1

for i in range(len(df_pos.columns)):
    for j in range(len(df_pos.columns)):
        if not (i==j):
            
            #辺の追加
            G.add_edge(df_pos.columns[i], df_pos.columns[j])
            #エッジの重みの追加
            if num_pre<len(G.edges):
                num_pre = len(G.edges)
                weight = 0
                if(df_pos.columns[i] in df_tr.columns) and (df_pos.columns[j] in df_tr.index):
                    if df_tr[df_pos.columns[i]][df_pos.columns[j]]:
                        weight = df_tr[df_pos.columns[i]][df_pos.columns[j]] * size
                elif(df_pos.columns[j] in df_tr.columns) and (df_pos.columns[i] in df_tr.index):
                    if df_tr[df_pos.columns[j]][df_pos.columns[i]]:
                        weight = df_tr[df_pos.columns[j]][df_pos.columns[i]] * size
            
            edge_weights.append(weight)
            
            
#座標の設定
pos={}
for i in range(len(df_pos.columns)):
    node = df_pos.columns[i]
    pos[node] = (df_pos[node][0], df_pos[node][1])
    
#描画
nx.draw(G, pos, with_labels=True, font_size=16, node_size=100, node_color='k', font_color='w', width=edge_weights)

plt.show()

In [ ]:
#制約条件計算関数

def condition_demand(df_tr, df_demand):
    flag = np.zeros(len(df_demand.columns))
    for i in range(len(df_demand.columns)):
        tmp_sum = sum(df_tr[df_demand.columns[i]])
        if(tmp_sum >= df_demand.iloc[0][i]):
            flag[i] = 1
    return flag


def condition_supply(df_tr, df_supply):
    flag = np.zeros(len(df_supply.columns))
    for i in range(len(df_supply.columns)):
        tmp_sum = sum(df_tr.loc[df_supply.columns[i]])
        if(tmp_sum <= df_supply.iloc[0][i]):
            flag[i] = 1
    return flag


print("需要条件計算結果" + str(condition_demand(df_tr_sol, df_demand)))
print("供給条件計算結果" + str(condition_supply(df_tr_sol, df_supply)))

In [ ]:
df_material = pd.read_csv("../input/100honch7/product_plan_material.csv", index_col="製品")
print(df_material)
df_profit = pd.read_csv("../input/100honch7/product_plan_profit.csv", index_col="製品")
print(df_profit)
df_stock = pd.read_csv("../input/100honch7/product_plan_stock.csv", index_col="項目")
print(df_stock)
df_plan = pd.read_csv("../input/100honch7/product_plan.csv", index_col="製品")
print(df_plan)

In [ ]:
#総利益を計算する関数
#商品1つ当たりの利益×商品の生産量、の和をとっていくことで計算できる
def product_plan(df_profit, df_plan):
    profit = 0
    for i in range(len(df_profit.index)):
        for j in range(len(df_plan.columns)):
            profit += df_profit.iloc[i][j] * df_plan.iloc[i][j]
    return profit

print("総利益:" + str(product_plan(df_profit,df_plan)))

In [ ]:
from ortoolpy import model_max

df = df_material.copy()
inv = df_stock

m = model_max()
v1 = {(i):LpVariable("v%d"%(i), lowBound=0) for i in  range(len(df_profit))} #製品数を同じ次元の変数v1を生成
m += lpSum(df_profit.iloc[i]*v1[i] for i in range(len(df_profit)))
for i in range(len(df_material.columns)):
    m += lpSum(df_material.iloc[j,i]*v1[j] for j in range(len(df_profit))) <= df_stock.iloc[:,i]
m.solve()

In [ ]:
def condition_stock(df_plan, df_material, df_stock):
    flag = np.zeros(len(df_material.columns))
    for i in range(len(df_material.columns)):
        temp_sum = 0
        for j in range(len(df_material.index)):
            temp_sum = temp_sum + df_material.iloc[j][i] * float(df_plan.iloc[j])
        if(temp_sum <= float(df_stock.iloc[0][i])):
            flag[i] = 1
        print(df_material.columns[i] + "　使用量:" + str(temp_sum) + ", 在庫:" + str(float(df_stock.iloc[0][i])))
    return flag

In [ ]:
print("制約条件計算結果:" + str(condition_stock(df_plan, df_material, df_stock)))

In [ ]:
製品 = list('AB')
需要地 = list('PQ')
工場 = list('XY')
レーン = (2,2)

#輸送費表
tbdi = pd.DataFrame(((j,k) for j in 需要地 for k in 工場), columns=['需要地','工場'])
tbdi['輸送費'] = [1,2,3,1]
print(tbdi)

#需要票
tbde = pd.DataFrame(((i,j) for j in 需要地 for i in 製品), columns=['需要地','製品'])
tbde['需要'] = [10,10,20,20]
print(tbde)

#生産表
tbfa = pd.DataFrame(((k,l,i,0,np.inf) for k,nl in zip (工場,レーン) for l in range(nl) for i in 製品), 
                    columns=['工場','レーン','製品','下限','上限'])
tbfa['生産費'] = [1, np.nan, np.nan, 1, 3, np.nan, 5, 3]
tbfa.dropna(inplace=True)
tbfa.loc[4,'上限']=10
print(tbfa)

from ortoolpy import logistics_network
_, tbdi2, _ = logistics_network(tbde, tbdi, tbfa)
print(tbfa)
print(tbdi2)

In [ ]:
print(tbdi2)
trans_cost = 0
for i in range(len(tbdi2.index)):
    trans_cost += tbdi2["輸送費"].iloc[i] * tbdi2["ValX"].iloc[i]
print("総輸送コスト:" + str(trans_cost))